In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
def MSE(y, tx, w):
    """Compute MSE at w
    
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,D+1)
        w: numpy array of shape=(D+1, ). The vector of model parameters.
        
    Returns:
        Returns the mean square error at w for input tx and output y
    """
    e = y - tx.dot(w)
    return np.mean(e**2)

In [3]:
def compute_gradient(y, tx, w):
    """Computes the gradient at w.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,D+1)
        w: numpy array of shape=(D+1, ). The vector of model parameters.
        
    Returns:
        An numpy array of shape (D+1, ) (same shape as w), containing the gradient of the loss at w.
    """
    e = y - tx.dot(w)
    return -tx.T.dot(e)/len(y)

In [4]:
def compute_gradientSGD(y, tx, w):
    """Computes the gradient SGD at w for batches of size one.
        
    Args:
        y: a number
        tx: numpy array of shape=(D+1, )
        w: numpy array of shape=(D+1, ). The vector of model parameters.
        
    Returns:
        An numpy array of shape (D+1, ) (same shape as w), containing the gradient of the loss at w.
    """
    e = y - tx.dot(w)
    return -tx.T.dot(e)

In [5]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """The Gradient Descent (GD) algorithm for least squares.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,D+1)
        initial_w: numpy array of shape=(D+1, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize
        
    Returns:
        w: the model parameter as numpy arrays of shape (2, ), for the last iteration of GD 
        loss: the loss value corresponding to w
    """
    
    w = initial_w
    for n_iter in range(max_iters):
        grad = compute_gradient(y, tx, w)
        w = w - gamma * grad

    loss = MSE(y, tx, w)
    return w, loss

In [6]:
def get_random_sample(y, tx):
    """get a random sample of (y, tx)
    
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,D+1)
        
    Returns:
        y_sample: a random sample of y as a number
        tx_sample: a random sample of tx as numpy arrays of shape (D+1, )
    """
    random_sample_index = np.random.randint(len(y))
    y_sample  = y [random_sample_index]
    tx_sample = tx[random_sample_index]
    return y_sample, tx_sample

In [7]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """The Stochastic Gradient Descent (SGD) algorithm for least squares using batches of size one.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,D+1)
        initial_w: numpy array of shape=(D+1, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize
        
    Returns:
        w: the model parameter as numpy arrays of shape (2, ), for the last iteration of SGD 
        loss: the loss value corresponding to w
    """
    
    w = initial_w
    
    for n_iter in range(max_iters):
        y_sample, tx_sample = get_random_sample(y, tx)
        grad = compute_gradientSGD(y_sample, tx_sample, w)
        w = w - gamma * grad
        
    loss = MSE(y, tx, w)
    return w, loss

Let's try to use the QR decomposition for more robust solving

In [8]:
def least_squares(y : np.array, tx : np.array):
    """Calculate the least squares solution.
       returns mse, and optimal weights.
    
    Args:
        y: numpy array of shape (N,), N is the number of samples.
        tx: numpy array of shape (N,D), D is the number of features.
    
    Returns:
        w: optimal weights, numpy array of shape(D,), D is the number of features.
        loss: loss value as a float
    """
    
    Q, R = np.linalg.qr(tx)
    w = np.linalg.solve(R, Q.T.dot(y))
    loss = MSE(y, tx, w)
    
    return w, loss

In [9]:
def ridge_regression(y : np.array, tx: np.array , lambda_):
    """Calculate the least squares solution with regularization parameter.
       returns mse, and optimal weights.
       
    Args:
        y: numpy array of shape (N,), N is the number of samples.
        tx: numpy array of shape (N,D), D is the number of features.
        lambda_: the regularization parameter as a scalar.
    
    Returns:
        w: optimal weights, numpy array of shape(D,), D is the number of features.
        loss: loss value as a float
    """
    
    D = tx.shape[1]
    lambda_I = np.eye(D) * np.sqrt(2*len(y)*lambda_)
    tx_expended = np.append(tx, lambda_I, axis=0)
    y_expended  = np.append(y, np.zeros(D))
    
    Q, R = np.linalg.qr(tx_expended)
    w = np.linalg.solve(R, Q.T.dot(y_expended))
    loss = MSE(y, tx, w)
    
    return w, loss

In [42]:
def ridge_regression_rolan(y : np.array, tx: np.array , lambda_: np.array ):
    """Ridge regression implementation using linear solver from numpy instead of inverse for performance reasons.
        (X.T*X + lambda*I)*w = X.t*y
    Args:
        y (np.array): The target values Y
        tx (np.array): The input array x padded with a vector of ones for the bias.
        lambda_ (np.array): The regularization parameter

    Returns:
        Tuple[np.array, float]: Weights w as a numpy array, loss value as a float
    """

    w = np.linalg.solve(tx.T.dot(tx) +  2*tx.shape[0]*lambda_ * np.eye(tx.shape[1]), tx.T.dot(y)) 
    loss = ((y-w.dot(tx.T))**2).mean(axis=0)
    
    
    return w, loss

In [27]:
def ridge_regression_cours(y, tx, lambda_):
    """implement ridge regression."""
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

# Sandbox for testing

In [10]:
x = [[12,16,71,99,45,27,80,58,4,50],
     [35,78,73,3,55,43,56,98,32,40]]
x = (x-np.mean(x, axis=1).reshape(2,1))/np.std(x, axis=1).reshape(2,1)
y = [56,22,37,78,83,55,70,94,12,40]
x = np.array(x).T
tx = np.insert(x, 0, 1, axis=1)
y = np.array(y)
linreg = LinearRegression().fit(x,y)
initial_w = np.array([0,0,0])
w = initial_w


print(np.append(np.array(linreg.intercept_), linreg.coef_))

[54.7        15.08492696  3.55532095]


In [21]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(y, tx.dot(w)))
print(MSE(y, tx, w))

3646.7
3646.7


### Testing for least Squares GD

In [11]:
least_squares_GD(y, tx, initial_w, 100, 0.1)

(array([54.69854709, 15.08425522,  3.55477245]), 423.1618830220783)

### Testing for least Squares SGD

In [12]:
least_squares_SGD(y, tx, initial_w, 100000, 0.0001)

(array([54.69782865, 15.08606612,  3.60861244]), 423.1647163135561)

In [13]:
least_squares(y, tx)

(array([54.7       , 15.08492696,  3.55532095]), 423.16188021914616)

In [25]:
ridge_regression(y, tx , lambda_=0)

(array([27.35      ,  7.45729914,  1.4666486 ]), 1231.1292309891364)

In [36]:
ridge_regression_cours(y, tx, lambda_=1)

array([18.23333333,  4.95639555,  0.90978386])

In [43]:
ridge_regression_rolan(y, tx, lambda_=1)

(array([18.23333333,  4.95639555,  0.90978386]), 1858.1954078615443)